# Bachlor seminar: 
## Public Charging Infrastructure for an Electric Car Sharing System
### Does the Infrastructure meet the Demand?
#### Imke Schwenke, 7351430


##### Importing Packages

In [2]:
import pandas as pd
import numpy as np
import folium

###### Importing data
retrieved from https://www.bundesnetzagentur.de/DE/Fachthemen/ElektrizitaetundGas/E-Mobilitaet/start.html

already filtered in Excel and then converted to csv

In [3]:
chargers_clg = pd.read_csv("Ladesaeulenregister.csv",';')
chargers_clg.head()

,Betreiber,Straße,Hausnummer,Adresszusatz,Postleitzahl,Ort,Bundesland,Kreis/kreisfreie Stadt,Breitengrad,Längengrad,...,Public Key1,Steckertypen2,P2 [kW],Public Key2,Steckertypen3,P3 [kW],Public Key3,Steckertypen4,P4 [kW],Public Key4
0,LANXESS Deutschland GmbH,Kennedyplatz,1,NaN,50569,Köln,Nordrhein-Westfalen,Kreisfreie Stadt Köln,"50,937417","6,97069",...,NaN,AC Steckdose Typ 2,22.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,LANXESS Deutschland GmbH,Kennedyplatz,1,NaN,50569,Köln,Nordrhein-Westfalen,Kreisfreie Stadt Köln,"50,937417","6,97069",...,NaN,AC Steckdose Typ 2,22.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,LANXESS Deutschland GmbH,Kennedyplatz,1,NaN,50569,Köln,Nordrhein-Westfalen,Kreisfreie Stadt Köln,"50,937417","6,97069",...,NaN,AC Steckdose Typ 2,22.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,LANXESS Deutschland GmbH,Kennedyplatz,1,NaN,50569,Köln,Nordrhein-Westfalen,Kreisfreie Stadt Köln,"50,937417","6,97069",...,NaN,AC Steckdose Typ 2,22.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Rheinenergie AG,Johannisstraße,74,NaN,50667,Köln,Nordrhein-Westfalen,Kreisfreie Stadt Köln,"50,94498","6,960778",...,NaN,AC Steckdose Typ 2,22.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
chargers_clg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 292 entries, 0 to 291
Data columns (total 26 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Betreiber               292 non-null    object 
 1   Straße                  292 non-null    object 
 2   Hausnummer              292 non-null    object 
 3   Adresszusatz            5 non-null      object 
 4   Postleitzahl            292 non-null    int64  
 5   Ort                     292 non-null    object 
 6   Bundesland              292 non-null    object 
 7   Kreis/kreisfreie Stadt  292 non-null    object 
 8   Breitengrad             292 non-null    object 
 9   Längengrad              292 non-null    object 
 10  Inbetriebnahmedatum     292 non-null    object 
 11  Anschlussleistung       292 non-null    object 
 12  Art der Ladeeinrichung  292 non-null    object 
 13  Anzahl Ladepunkte       292 non-null    int64  
 14  Steckertypen1           292 non-null    ob

In [5]:
print(chargers_clg["Ort"].unique())
print(chargers_clg["Bundesland"].unique())
print(chargers_clg["Kreis/kreisfreie Stadt"].unique())
print(chargers_clg["Adresszusatz"].unique())

['Köln' 'Köln Deutz' 'Köln-Pesch' 'Köln-Poll' 'Köln-Merheim']
['Nordrhein-Westfalen']
['Kreisfreie Stadt Köln']
[nan 'ggü.' '(Auenweg)' 'A']


In [6]:
print(chargers_clg["Art der Ladeeinrichung"].unique())

['Normalladeeinrichtung' 'Schnellladeeinrichtung']


##### Data cleaning
As we consider the whole city of Cologne, we do not need the column 'Ort'. We work with the coordinates ('Breitengrad','Längengrad'). Other information about the location is not necessary. We only focus on if the station is a fast or a slow charging station. This means we just include the column 'Art der Ladeeinrichtung'. It should be noted that each station has different connectors, a slow and a fast charging one. However, we concentrate on the given classification.

In [7]:
chargers_clg_shortend = chargers_clg.drop(columns=["Straße","Hausnummer","Adresszusatz","Postleitzahl","Ort","Bundesland", "Kreis/kreisfreie Stadt","Inbetriebnahmedatum","Anschlussleistung","Steckertypen1","P1 [kW]","Public Key1","Steckertypen2","P2 [kW]","Public Key2","Steckertypen3","P3 [kW]","Public Key3","Steckertypen4","P4 [kW]","Public Key4"])
chargers_clg_shortend.head()


,Betreiber,Breitengrad,Längengrad,Art der Ladeeinrichung,Anzahl Ladepunkte
0,LANXESS Deutschland GmbH,"50,937417","6,97069",Normalladeeinrichtung,2
1,LANXESS Deutschland GmbH,"50,937417","6,97069",Normalladeeinrichtung,2
2,LANXESS Deutschland GmbH,"50,937417","6,97069",Normalladeeinrichtung,2
3,LANXESS Deutschland GmbH,"50,937417","6,97069",Normalladeeinrichtung,2
4,Rheinenergie AG,"50,94498","6,960778",Normalladeeinrichtung,2


We also add a column 'Koordinaten' ('Coordinates') to the dataframe. 

In [8]:
chargers_clg_shortend['Breitengrad'] = chargers_clg_shortend['Breitengrad'].str.replace(',', '.')
chargers_clg_shortend['Längengrad'] = chargers_clg_shortend['Längengrad'].str.replace(',', '.')
chargers_clg_shortend['Breitengrad'] = chargers_clg_shortend['Breitengrad'].astype(float)
chargers_clg_shortend['Längengrad'] = chargers_clg_shortend['Längengrad'].astype(float)
chargers_clg_shortend["Koordinaten"] = list(zip(chargers_clg_shortend["Breitengrad"].round(4),chargers_clg_shortend["Längengrad"].round(4)))
chargers_clg_shortend.head()

,Betreiber,Breitengrad,Längengrad,Art der Ladeeinrichung,Anzahl Ladepunkte,Koordinaten
0,LANXESS Deutschland GmbH,50.937417,6.970690,Normalladeeinrichtung,2,"(50.9374, 6.9707)"
1,LANXESS Deutschland GmbH,50.937417,6.970690,Normalladeeinrichtung,2,"(50.9374, 6.9707)"
2,LANXESS Deutschland GmbH,50.937417,6.970690,Normalladeeinrichtung,2,"(50.9374, 6.9707)"
3,LANXESS Deutschland GmbH,50.937417,6.970690,Normalladeeinrichtung,2,"(50.9374, 6.9707)"
4,Rheinenergie AG,50.944980,6.960778,Normalladeeinrichtung,2,"(50.945, 6.9608)"


In [9]:
chargers_clg_shortend.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 292 entries, 0 to 291
Data columns (total 6 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Betreiber               292 non-null    object 
 1   Breitengrad             292 non-null    float64
 2   Längengrad              292 non-null    float64
 3   Art der Ladeeinrichung  292 non-null    object 
 4   Anzahl Ladepunkte       292 non-null    int64  
 5   Koordinaten             292 non-null    object 
dtypes: float64(2), int64(1), object(3)
memory usage: 13.8+ KB


##### Creating a map for charger location
We now create a map of cologne where each slow chargers is represented in a red circle and each fast charger in a blue one.

In [10]:


charger_points_map = folium.Map(location=(50.9413, 6.9583),  tiles='Stamen Toner', 
                       zoom_start=10, control_scale=True, max_zoom=20)


charger_points_slow = chargers_clg_shortend[chargers_clg_shortend["Art der Ladeeinrichung"]=='Normalladeeinrichtung']
charger_points_fast = chargers_clg_shortend[chargers_clg_shortend["Art der Ladeeinrichung"]=='Schnellladeeinrichtung']

positions = list(charger_points_slow["Koordinaten"].unique())
for position in positions:
    folium.CircleMarker(radius=10, location=position, popup='The Waterfront', 
                                 color='crimson', fill_color='crimson').add_to(charger_points_map)
    
positions = list(charger_points_fast["Koordinaten"].unique())
for position in positions:
    folium.CircleMarker(radius=10, location=position, popup='The Waterfront', 
                                 color='blue', fill_color='blue').add_to(charger_points_map)
    
charger_points_map

In [14]:

charger_points_map.save('cologne_center.html')

In [13]:
print(charger_points_slow.count())
print('-------------------------------------------------------------------')
print(charger_points_fast.count())

Betreiber                 269
Breitengrad               269
Längengrad                269
Art der Ladeeinrichung    269
Anzahl Ladepunkte         269
Koordinaten               269
dtype: int64
-------------------------------------------------------------------
Betreiber                 23
Breitengrad               23
Längengrad                23
Art der Ladeeinrichung    23
Anzahl Ladepunkte         23
Koordinaten               23
dtype: int64
